In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Local development and docker image components

- This section assumes that you have already created a program to perform the task required in a particular step of your ML workflow. This example uses an MNIST model training script.

- Then, this example packages your program as a Docker container image.

- Then, this example calls kfp.components.ContainerOp to convert it to a Kubeflow pipeline component.

Note: Ensure that you have Docker installed, if you want to build the image locally, by running the following command:
 
`which docker`
 
The result should be something like:

`/usr/bin/docker`

In [1]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp
import datetime

import kubernetes as k8s

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/kubeflow-pipeline-fantasy.json'

In [4]:
# Required Parameters
PROJECT_ID='kubeflow-pipeline-fantasy'
GCS_BUCKET='gs://kubeflow-pipeline-ptt'

## Create client

If you run this notebook **outside** of a Kubeflow cluster, run the following command:
- `host`: The URL of your Kubeflow Pipelines instance, for example "https://`<your-deployment>`.endpoints.`<your-project>`.cloud.goog/pipeline"
- `client_id`: The client ID used by Identity-Aware Proxy
- `other_client_id`: The client ID used to obtain the auth codes and refresh tokens.
- `other_client_secret`: The client secret used to obtain the auth codes and refresh tokens.

```python
client = kfp.Client(host, client_id, other_client_id, other_client_secret)
```

If you run this notebook **within** a Kubeflow cluster, run the following command:
```python
client = kfp.Client()
```

You'll need to create OAuth client ID credentials of type `Other` to get `other_client_id` and `other_client_secret`. Learn more about [creating OAuth credentials](
https://cloud.google.com/iap/docs/authentication-howto#authenticating_from_a_desktop_app)

In [5]:
# Optional Parameters, but required for running outside Kubeflow cluster

# # The host for full deployment of Kubeflow ends with '/pipeline'
# HOST = 'https://kubeflow-st-ui.endpoints.kubeflow-pipeline-fantasy.cloud.goog/pipeline'
# # Full deployment of Kubeflow on GCP is usually protected through IAP, therefore the following 
# # will be needed to access the endpoint
# CLIENT_ID = "493831447550-os23o55235htd9v45a9lsejv8d1plhd0.apps.googleusercontent.com"
# OTHER_CLIENT_ID = "493831447550-iu24vv6id3ng5smhf2lboovv5qukuhbh.apps.googleusercontent.com"
# OTHER_CLIENT_SECRET = "cB8Xj-rb9JWCYcCRDlpTMfhc"

# The host for managed 'AI Platform Pipeline' ends with 'pipelines.googleusercontent.com'
HOST = 'https://69a95965149a4145-dot-asia-east1.pipelines.googleusercontent.com'

In [6]:
# This is to ensure the proper access token is present to reach the end point for managed 'AI Platform Pipeline'
# If you are not working with managed 'AI Platform Pipeline', this step is not necessary
! gcloud auth print-access-token

ya29.c.KoAB2Aeb2BSyRb_xo_WewV61G-IqBbsmHHC1cNOh5OLRilh5TwlX45-iDQ6FSRCYVrlnaIGM8Gd8ZiL24WT-z5CR_M2e2wvP1hXB6FC7iKVv-ZWY0eaUWkwPuNty-WtnzTI_byUZnzf6-cnlESNys6Mje7d2hs89OlfdFFItawLznjw


In [7]:
# Create kfp client
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    if HOST.endswith('googleusercontent.com'):
        CLIENT_ID = None
        OTHER_CLIENT_ID = None
        OTHER_CLIENT_SECRET = None

    client = kfp.Client(host=HOST, 
                        client_id=CLIENT_ID,
                        other_client_id=OTHER_CLIENT_ID, 
                        other_client_secret=OTHER_CLIENT_SECRET)

## Wrap an existing Docker container image using `ContainerOp`

### Writing the program code

The following cell creates a file `app.py` that contains a Python script. The script downloads MNIST dataset, trains a Neural Network based classification model, writes the training log and exports the trained model to Google Cloud Storage.

Your component can create outputs that the downstream components can use as inputs. Each output must be a string and the container image must write each output to a separate local text file. For example, if a training component needs to output the path of the trained model, the component writes the path into a local file, such as `/output.txt`.

In [52]:
%%bash

# Create folders if they don't exist.
mkdir -p tmp/components/mnist_training

# Create the Python file that lists GCS blobs.
cat > ./tmp/components/mnist_training/app.py <<HERE
import argparse
from datetime import datetime
import tensorflow as tf
import json

gfile = tf.io.gfile

parser = argparse.ArgumentParser()
parser.add_argument(
    '--model_file', type=str, required=True, help='Name of the model file.')
parser.add_argument(
    '--bucket', type=str, required=True, help='GCS bucket name.')
args = parser.parse_args()

bucket=args.bucket
model_file=args.model_file

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())    

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir=bucket + '/logs/' + datetime.now().date().__str__()),
  # Interrupt training if val_loss stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
]

history = model.fit(x_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(x_test, y_test))

metrics = {
    'metrics': [
        {
            'name': 'accuracy', # The name of the metric. Visualized as the column name in the runs table.
            'numberValue':  float(history.history['accuracy'][-1]), # The value of the metric. Must be a numeric value.
            'format': "PERCENTAGE",
        },
        {
            'name': 'val-accuracy', # The name of the metric. Visualized as the column name in the runs table.
            'numberValue':  float(history.history['val_accuracy'][-1]), # The value of the metric. Must be a numeric value.
            'format': "PERCENTAGE",
        }
    ]
}
with open('/mlpipeline-metrics.json', 'w') as f:
    json.dump(metrics, f)

model.save(model_file)

gcs_path = bucket + "/" + model_file

if gfile.exists(gcs_path):
    gfile.remove(gcs_path)

gfile.copy(model_file, gcs_path)
with open('/output.txt', 'w') as f:
  f.write(gcs_path)
HERE

### Creating a Dockerfile

Now create a container that runs the script. Start by creating a Dockerfile. A Dockerfile contains the instructions to assemble a Docker image. The `FROM` statement specifies the Base Image from which you are building. `WORKDIR` sets the working directory. When you assemble the Docker image, `COPY` copies the required files and directories (for example, `app.py`) to the file system of the container. `RUN` executes a command (for example, install the dependencies) and commits the results. 

In [53]:
%%bash

# Create Dockerfile.
cat > ./tmp/components/mnist_training/Dockerfile <<EOF
FROM tensorflow/tensorflow:2.1.0-py3
WORKDIR /app
COPY . /app
EOF

### Build docker image

Now that we have created our Dockerfile for creating our Docker image. Then we need to push the image to a registry to host the image. 
- We are going to use Cloud Build to build the image and push to the Container Registry (GCR).
- It is possible to build the `kfp.containers.build_image_from_working_dir` to build the image and push to the Container Registry (GCR), which uses [kaniko](https://cloud.google.com/blog/products/gcp/introducing-kaniko-build-container-images-in-kubernetes-and-google-container-builder-even-without-root-access).
- It is possible to build the image locally using Docker and then to push it to GCR.

**Note**:
If you run this notebook **within Kubeflow cluster**, **with Kubeflow version >= 0.7**, you need to ensure that valid credentials are created within your notebook's namespace.
- With Kubeflow version >= 0.7, the credential is supposed to be copied automatically while creating notebook through `Configurations`, which doesn't work properly at the time of creating this notebook. 
- You can also add credentials to the new namespace by either [copying credentials from an existing Kubeflow namespace, or by creating a new service account](https://www.kubeflow.org/docs/gke/authentication/#kubeflow-v0-6-and-before-gcp-service-account-key-as-secret).
- The following cell demonstrates how to copy the default secret to your own namespace.

```bash
%%bash

NAMESPACE=<your notebook name space>
SOURCE=kubeflow
NAME=user-gcp-sa
SECRET=$(kubectl get secrets \${NAME} -n \${SOURCE} -o jsonpath="{.data.\${NAME}\.json}" | base64 -D)
kubectl create -n \${NAMESPACE} secret generic \${NAME} --from-literal="\${NAME}.json=\${SECRET}"
```

In [54]:
IMAGE_NAME="mnist_training_kf_pipeline"
TAG="latest" # "v_$(date +%Y%m%d_%H%M%S)"

GCR_IMAGE="gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}".format(
    PROJECT_ID=PROJECT_ID,
    IMAGE_NAME=IMAGE_NAME,
    TAG=TAG
)

APP_FOLDER='./tmp/components/mnist_training/'

In [55]:
if HOST.endswith('googleusercontent.com'):
    # kaniko is not pre-installed with managed "AI Platform Pipeline"
    import subprocess
    # ! gcloud builds submit --tag ${IMAGE_NAME} ${APP_FOLDER}
    cmd = ['gcloud', 'builds', 'submit', '--tag', GCR_IMAGE, APP_FOLDER]
    build_log = (subprocess.run(cmd, stdout=subprocess.PIPE).stdout[:-1].decode('utf-8'))
    print(build_log)

    import re
    m = re.search(r'latest: digest: sha256:.* size', build_log)
    digest = m.group(0).split(' ')[2]
    
    image_name = "gcr.io/{PROJECT_ID}/{IMAGE_NAME}@{DIGEST}".format(
        PROJECT_ID=PROJECT_ID,
        IMAGE_NAME=IMAGE_NAME,
        DIGEST=digest
    )
    
else:
    if kfp.__version__ <= '0.1.36':
        # kfp with version 0.1.36+ introduce broken change that will make the following code not working'
        import subprocess
        CLUSTER_NAME=''
        ZONE=''
        if CLUSTER_NAME and ZONE:
            # ! gcloud container clusters get-credentials ${CLUSTER_NAME} --region ${ZONE}
            cmd = ['gcloud', 'container', 'clusters', 'get-credentials', CLUSTER_NAME, ZONE]
            update_kubeconfig= (subprocess.run(cmd, stdout=subprocess.PIPE).stdout[:-1].decode('utf-8'))
            print(update_kubeconfig)
        
        builder = kfp.containers._container_builder.ContainerBuilder(
            gcs_staging=GCS_BUCKET + "/kfp_container_build_staging"
        )

        image_name = kfp.containers.build_image_from_working_dir(
            image_name=GCR_IMAGE,
            working_dir=APP_FOLDER,
            builder=builder
        )
    else:
        raise("Please build the docker image use either [Docker] or [Cloud Build]")

print(image_name)

----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "b53df073-55c5-494e-8b37-e9cf7645acb1"

FETCHSOURCE
Fetching storage object: gs://kubeflow-pipeline-fantasy_cloudbuild/source/1597600022.4-a65bbbfc8e114d9895793a02808c92e0.tgz#1597600023280109
Copying gs://kubeflow-pipeline-fantasy_cloudbuild/source/1597600022.4-a65bbbfc8e114d9895793a02808c92e0.tgz#1597600023280109...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker

                   ***** NOTICE *****

Alternative official `docker` images, including multiple versions across
multiple platforms, are maintained by the Docker Team. For details, please
visit https://hub.docker.com/_/docker.

                ***** END OF NOTICE *****
Sending build context to Docker daemon   5.12kB
Step 1/3 : FROM tensorflow/tensorf

#### If you want to use docker to build the image
Run the following in a cell
```bash
%%bash -s "{PROJECT_ID}"

IMAGE_NAME="mnist_training_kf_pipeline"
TAG="latest" # "v_$(date +%Y%m%d_%H%M%S)"

# Create script to build docker image and push it.
cat > ./tmp/components/mnist_training/build_image.sh <<HERE
PROJECT_ID="${1}"
IMAGE_NAME="${IMAGE_NAME}"
TAG="${TAG}"
GCR_IMAGE="gcr.io/\${PROJECT_ID}/\${IMAGE_NAME}:\${TAG}"
docker build -t \${IMAGE_NAME} .
docker tag \${IMAGE_NAME} \${GCR_IMAGE}
docker push \${GCR_IMAGE}
docker image rm \${IMAGE_NAME}
docker image rm \${GCR_IMAGE}
HERE

cd tmp/components/mnist_training
bash build_image.sh
```

### Define each component
Define a component by creating an instance of `kfp.dsl.ContainerOp` that describes the interactions with the Docker container image created in the previous step. You need to specify 
- component name
- the image to use
- the command to run after the container starts (If None, uses default CMD in defined in container.)
- the input arguments
- the file outputs (In the `app.py` above, the path of the trained model is written to `/output.txt`.)

In [56]:
def mnist_train_op(model_file, bucket):
    return dsl.ContainerOp(
      name="mnist_training_container",
      image=image_name,
      command=['python', '/app/app.py'],
      file_outputs={
          'outputs': '/output.txt',
          'mlpipeline-metrics': '/mlpipeline-metrics.json'
      },
      arguments=['--bucket', bucket, '--model_file', model_file]
    )

Optionally, you can use the kfp.dsl.component decorator to enable static type checking in the DSL compiler.
```python
@kfp.dsl.component
def mnist_train_op(model_file: kfp.dsl.types.String(), 
                   bucket: kfp.dsl.types.String()) -> kfp.dsl.types.GCSPath():
    return dsl.ContainerOp(
      name="mnist_training_container",
      image='gcr.io/{}/mnist_training_kf_pipeline:latest'.format(PROJECT_ID),
      command=['python', '/app/app.py'],
      file_outputs={'outputs': '/output.txt'},
      arguments=['--bucket', bucket, '--model_file', model_file]
    )
```

### Create your workflow as a Python function

Define your pipeline as a Python function. ` @kfp.dsl.pipeline` is a required decoration including `name` and `description` properties. Then compile the pipeline function. After the compilation is completed, a pipeline file is created.

In [57]:
# Define the pipeline
@dsl.pipeline(
   name='Mnist pipeline',
   description='A toy pipeline that performs mnist model training.'
)
def mnist_container_pipeline(
    model_file: str = 'mnist_model.h5', 
    bucket: str = GCS_BUCKET
):
    mnist_train_op(model_file=model_file, bucket=bucket) # .apply(gcp.use_gcp_secret('user-gcp-sa'))

### Submit a pipeline run

In [58]:
pipeline_func = mnist_container_pipeline

In [59]:
experiment_name = 'minist_kubeflow'

arguments = {"model_file":"mnist_model.h5",
             "bucket":GCS_BUCKET}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

**As an alternative, you can compile the pipeline into a package.** The compiled pipeline can be easily shared and reused by others to run the pipeline.

```python
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

experiment = client.create_experiment('python-functions-mnist')

run_result = client.run_pipeline(
    experiment_id=experiment.id, 
    job_name=run_name, 
    pipeline_package_path=pipeline_filename, 
    params=arguments)
```